## Homework: Introduction

In this homework, we'll learn more about search and use Elastic Search for practice. 

> It's possible that your answers won't match exactly. If it's the case, select the closest one.

## Q1. Running Elastic 

Run Elastic Search 8.17.6, and get the cluster information. If you run it on localhost, this is how you do it:

```bash
curl localhost:9200
```

What's the `version.build_hash` value?

### Launching the Elasticsearch server locally with Docker

In [3]:
#!docker pull docker.elastic.co/elasticsearch/elasticsearch:8.4.3
#!docker run -d --name elasticsearch -p 9200:9200 -p 9300:9300 -e "discovery.type=single-node" -e "xpack.security.enabled=false" docker.elastic.co/elasticsearch/elasticsearch:8.4.3

c1f6b0439a84f9c9b95aac4d930df0b09b4343a33ae0a8eb13192608de230785


### Testing the server

In [1]:
!curl localhost:9200

{
  "name" : "62f024542ac4",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "b-pkltLAT0mMBKAp--CG4Q",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   539  100   539    0     0  64067      0 --:--:-- --:--:-- --:--:-- 67375


## Getting the data

Now let's get the FAQ data. You can run this snippet:

In [2]:
!pip install requests -q


[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import requests 
from rich import print as prrint

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

prrint(documents[0])

{
    'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and 
hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' 
live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts
using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in 
DataTalks.Club's Slack and join the channel.",
    'section': 'General course-related questions',
    'question': 'Course - When will the course start?',
    'course': 'data-engineering-zoomcamp'
}

Note that you need to have the `requests` library:

```bash
pip install requests
```

## Q2. Indexing the data

Index the data in the same way as was shown in the course videos. Make the `course` field a keyword and the rest should be text. 

Don't forget to install the ElasticSearch client for Python:


In [4]:
!pip install elasticsearch==8.4.1 elastic-transport==8.4.1 -q


[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Which function do you use for adding your data to elastic?

* `insert`
* `index`
* `put`
* `add`

In [3]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch("http://localhost:9200")
info = es_client.info()
prrint(info)

{'name': '62f024542ac4', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'b-pkltLAT0mMBKAp--CG4Q', 'version': 
{'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 
'42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': 
False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 
'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}

In [4]:
index_settings = {
    "settings":{
        "number_of_shards": 1,
        "number_of_replicas":0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"}
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

C:\Users\Alexander\AppData\Local\Temp\ipykernel_21100\2938790325.py:18: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es_client.indices.create(index=index_name, body=index_settings)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [5]:
from tqdm.auto import tqdm

for document in tqdm(documents):
    es_client.index(index=index_name, document=document)

d:\Program Files\Python310\enviroments\llmenvs\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 948/948 [02:10<00:00,  7.29it/s]


## Q3. Searching

Now let's search in our index. 

We will execute a query `"How do execute a command on a Kubernetes pod?"`. 

Use only `question` and `text` fields and give `question` a boost of 4, and use `"type": "best_fields"`.

What's the score for the top ranking result?

* 84.50
* 64.50
* 44.50
* 24.50

Look at the `_score` field.

In [6]:
query = "How do execute a command on a Kubernetes pod?"

search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
        }
    }
}

response = es_client.search(index=index_name, body=search_query)

for hit in response["hits"]["hits"]:
    print(hit["_score"])

C:\Users\Alexander\AppData\Local\Temp\ipykernel_21100\925483546.py:18: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es_client.search(index=index_name, body=search_query)


44.50556
35.433445
33.70974
33.2635
32.589073


## Q4. Filtering

Now ask a different question: `"How do copy a file to a Docker container?"`.

This time we are only interested in questions from `machine-learning-zoomcamp`.

Return 3 results. What's the 3rd question returned by the search engine?

* How do I debug a docker container?
* How do I copy files from a different folder into docker container’s working directory?
* How do Lambda container images work?
* How can I annotate a graph?

In [7]:
query = "How do copy a file to a Docker container?"

search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

response = es_client.search(index=index_name, body=search_query)
for i, hit in enumerate(response["hits"]["hits"]):
    print(f"{i+1}.", hit['_source']['question'])

1. How do I debug a docker container?
2. How do I copy files from my local machine to docker container?
3. How do I copy files from a different folder into docker container’s working directory?


C:\Users\Alexander\AppData\Local\Temp\ipykernel_21100\2407438344.py:23: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es_client.search(index=index_name, body=search_query)


## Q5. Building a prompt

Now we're ready to build a prompt to send to an LLM. 

Take the records returned from Elasticsearch in Q4 and use this template to build the context. Separate context entries by two linebreaks (`\n\n`)

```python
context_template = """
Q: {question}
A: {text}
""".strip()
```

Now use the context you just created along with the `"How do copy a file to a Docker container?"` question 
to construct a prompt using the template below:

```
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()
```

What's the length of the resulting prompt? (use the `len` function)

* 946
* 1446
* 1946
* 2446

In [8]:
context_template = """
Q: {question}
A: {text}
""".strip()

contexts = []

for i, hit in enumerate(response["hits"]["hits"]):
    question = hit['_source']['question']
    text = hit['_source']['text']
    entry = context_template.format(question=question, text=text)
    contexts.append(entry)

context = "\n\n".join(contexts)

query = "How do copy a file to a Docker container?"

prompt_template = f"""
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

len(prompt_template)

1491

## Q6. Tokens

When we use the OpenAI Platform, we're charged by the number of 
tokens we send in our prompt and receive in the response.

The OpenAI python package uses `tiktoken` for tokenization:

```bash
pip install tiktoken
```

Let's calculate the number of tokens in our query: 

```python
encoding = tiktoken.encoding_for_model("gpt-4o")
```

Use the `encode` function. How many tokens does our prompt have?

* 120
* 220
* 320
* 420

Note: to decode back a token into a word, you can use the `decode_single_token_bytes` function:

```python
encoding.decode_single_token_bytes(63842)
```

In [41]:
!pip install tiktoken -q


[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o")
tokens = encoding.encode(prompt_template)
print(f"Number of tokens: {len(tokens)}")

Number of tokens: 326


## Bonus: generating the answer (ungraded)

Let's send the prompt to OpenAI. What's the response?  

Note: you can replace OpenAI with Ollama. See module 2.

In [46]:
!pip install ollama -q


[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
#Bash command: ollama run llama3
!curl http://localhost:11434

Ollama is running


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    17  100    17    0     0     81      0 --:--:-- --:--:-- --:--:--    81


In [ ]:
import ollama

response = ollama.chat(
    model='llama3',
    messages=[{"role": "user", "content": prompt_template}]
)

print("\033[1m\033[92m"+"PROMPT "+"+"*125+"\033[0m")
print(prompt_template)
print("\n\033[1m\033[92m"+"OLLAMA "+"+"*125+"\033[0m")
print(response['message']['content'])

PROMPT +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do I copy files from a different folder into docker container’s working directory?

CONTEXT:
Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local mach

## Bonus: calculating the costs (ungraded)

Suppose that on average per request we send 150 tokens and receive back 250 tokens.

How much will it cost to run 1000 requests?

You can see the prices [here](https://openai.com/api/pricing/)

On June 17, the prices for gpt4o are:

* Input: $0.005 / 1K tokens
* Output: $0.015 / 1K tokens

You can redo the calculations with the values you got in Q6 and Q7.

In [24]:
tokens_prompt = encoding.encode(prompt_template)
input_tokens = len(tokens_prompt)

answer = response['message']['content']
tokens_answer = encoding.encode(answer)
output_tokens = len(tokens_answer)

input_price = (0.005/1e3)*(1e3*input_tokens)
output_price = (0.015/1e3)*(1e3*output_tokens)

print(f"Total price at June 17, 2024: ${round(input_price + output_price, 2)}")

input_price = (5/1e6)*(1e3*input_tokens)
output_price = (20/1e6)*(1e3*output_tokens)

print(f"Total price at June 13, 2025: ${round(input_price + output_price, 2)}")

Total price at June 17, 2024: $3.12
Total price at June 13, 2025: $3.61
